In [3]:
from collections import defaultdict
import json
import os
import pickle
import statistics

from requests import Session

import matplotlib.pyplot as plt
import pandas as pd

import statbotics

%matplotlib notebook

In [4]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

sb = statbotics.Statbotics()

In [5]:
def dump(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path, "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load(file):
    with open(file, "rb") as f:
        return pickle.load(f)


def dump_cache(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path + "/data.p", "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load_cache(file):
    with open(file + "/data.p", "rb") as f:
        return pickle.load(f)

    
    
def get_tba(url):
    if os.path.exists("../../backend/cache/" + url + "/data.p"):
        # Cache Hit
        return load_cache("../../backend/cache/" + url)

    response = session.get(read_prefix + url)
    data = response.json()

    # Cache Miss
    dump_cache("../../backend/cache/" + url, data)
    return data

In [6]:
events = get_tba("events/2023")
events = [(e["key"], e["week"]) for e in events]

print(len(events))

all_matches = []
for event, week in events:
    all_matches.extend([x, week] for x in get_tba("event/" + event + "/matches"))
    
print(len(all_matches))

179
10449


In [7]:
year = sb.get_year(2023)
teams = sb.get_team_years(year=2023, limit=10000)

TOTAL_MEAN, TOTAL_SD = year["score_mean"], year["score_sd"]
starting_epas = {t["team"]: (t["epa_start"] - TOTAL_MEAN / 3) / TOTAL_SD for t in teams}
locations = {t["team"]: (t["country"], t["state"], t["district"]) for t in teams}
print(starting_epas[254], starting_epas[5511])

0.9612851952770209 0.03689827429609454


In [12]:
def process_match(m, w):
    rb = m["score_breakdown"]["red"]
    bb = m["score_breakdown"]["blue"]
    
    out = {
        "week": w + 1, # TBA off by one
        "event": m["event_key"],
        "key": m["key"],
        "time": m["actual_time"],
        "playoff": m["comp_level"] != "qm",
        "red": [int(x[3:]) for x in m["alliances"]["red"]["team_keys"]],
        "blue": [int(x[3:]) for x in m["alliances"]["blue"]["team_keys"]],
        "red_auto_cycles": rb["autoGamePieceCount"],
        "blue_auto_cycles": bb["autoGamePieceCount"],
        "red_auto_points": rb["autoGamePiecePoints"],
        "blue_auto_points": bb["autoGamePiecePoints"],
        "red_teleop_cycles": rb["teleopGamePieceCount"],
        "blue_teleop_cycles": bb["teleopGamePieceCount"],
        "red_teleop_points": rb["teleopGamePiecePoints"],
        "blue_teleop_points": bb["teleopGamePiecePoints"],
        "red_link_points": rb["linkPoints"],
        "blue_link_points": bb["linkPoints"],
        "red_total_cycles": rb["autoGamePieceCount"] + rb["teleopGamePieceCount"],
        "blue_total_cycles": bb["autoGamePieceCount"] + bb["teleopGamePieceCount"],
        "red_total_points": rb["autoGamePiecePoints"] + rb["teleopGamePiecePoints"] + rb["linkPoints"],
        "blue_total_points": bb["autoGamePiecePoints"] + bb["teleopGamePiecePoints"] + bb["linkPoints"],
        "red_auto_bot_cubes": len([x for x in rb["autoCommunity"]["B"] if x == "Cube"]),
        "blue_auto_bot_cubes": len([x for x in bb["autoCommunity"]["B"] if x == "Cube"]),
        "red_auto_mid_cubes": len([x for x in rb["autoCommunity"]["M"] if x == "Cube"]),
        "blue_auto_mid_cubes": len([x for x in bb["autoCommunity"]["M"] if x == "Cube"]),
        "red_auto_top_cubes": len([x for x in rb["autoCommunity"]["T"] if x == "Cube"]),
        "blue_auto_top_cubes": len([x for x in bb["autoCommunity"]["T"] if x == "Cube"]),
        "red_auto_bot_cones": len([x for x in rb["autoCommunity"]["B"] if x == "Cone"]),
        "blue_auto_bot_cones": len([x for x in bb["autoCommunity"]["B"] if x == "Cone"]),
        "red_auto_mid_cones": len([x for x in rb["autoCommunity"]["M"] if x == "Cone"]),
        "blue_auto_mid_cones": len([x for x in bb["autoCommunity"]["M"] if x == "Cone"]),
        "red_auto_top_cones": len([x for x in rb["autoCommunity"]["T"] if x == "Cone"]),
        "blue_auto_top_cones": len([x for x in bb["autoCommunity"]["T"] if x == "Cone"]),
        "red_auto_bot_cycles": len([x for x in rb["autoCommunity"]["B"] if x != "None"]),
        "blue_auto_bot_cycles": len([x for x in bb["autoCommunity"]["B"] if x != "None"]),
        "red_auto_mid_cycles": len([x for x in rb["autoCommunity"]["M"] if x != "None"]),
        "blue_auto_mid_cycles": len([x for x in bb["autoCommunity"]["M"] if x != "None"]),
        "red_auto_top_cycles": len([x for x in rb["autoCommunity"]["T"] if x != "None"]),
        "blue_auto_top_cycles": len([x for x in bb["autoCommunity"]["T"] if x != "None"]),
        
        "red_teleop_bot_cubes": len([x for x in rb["teleopCommunity"]["B"] if x == "Cube"]),
        "blue_teleop_bot_cubes": len([x for x in bb["teleopCommunity"]["B"] if x == "Cube"]),
        "red_teleop_mid_cubes": len([x for x in rb["teleopCommunity"]["M"] if x == "Cube"]),
        "blue_teleop_mid_cubes": len([x for x in bb["teleopCommunity"]["M"] if x == "Cube"]),
        "red_teleop_top_cubes": len([x for x in rb["teleopCommunity"]["T"] if x == "Cube"]),
        "blue_teleop_top_cubes": len([x for x in bb["teleopCommunity"]["T"] if x == "Cube"]),
        "red_teleop_bot_cones": len([x for x in rb["teleopCommunity"]["B"] if x == "Cone"]),
        "blue_teleop_bot_cones": len([x for x in bb["teleopCommunity"]["B"] if x == "Cone"]),
        "red_teleop_mid_cones": len([x for x in rb["teleopCommunity"]["M"] if x == "Cone"]),
        "blue_teleop_mid_cones": len([x for x in bb["teleopCommunity"]["M"] if x == "Cone"]),
        "red_teleop_top_cones": len([x for x in rb["teleopCommunity"]["T"] if x == "Cone"]),
        "blue_teleop_top_cones": len([x for x in bb["teleopCommunity"]["T"] if x == "Cone"]),
        "red_teleop_bot_cycles": len([x for x in rb["teleopCommunity"]["B"] if x != "None"]),
        "blue_teleop_bot_cycles": len([x for x in bb["teleopCommunity"]["B"] if x != "None"]),
        "red_teleop_mid_cycles": len([x for x in rb["teleopCommunity"]["M"] if x != "None"]),
        "blue_teleop_mid_cycles": len([x for x in bb["teleopCommunity"]["M"] if x != "None"]),
        "red_teleop_top_cycles": len([x for x in rb["teleopCommunity"]["T"] if x != "None"]),
        "blue_teleop_top_cycles": len([x for x in bb["teleopCommunity"]["T"] if x != "None"]),
    }
    
    out["red_auto_cubes"] = out["red_auto_bot_cubes"] + out["red_auto_mid_cubes"] + out["red_auto_top_cubes"]
    out["blue_auto_cubes"] = out["blue_auto_bot_cubes"] + out["blue_auto_mid_cubes"] + out["blue_auto_top_cubes"]
    out["red_auto_cones"] = out["red_auto_bot_cones"] + out["red_auto_mid_cones"] + out["red_auto_top_cones"]
    out["blue_auto_cones"] = out["blue_auto_bot_cones"] + out["blue_auto_mid_cones"] + out["blue_auto_top_cones"]
    out["red_auto_cube_points"] = 3 * out["red_auto_bot_cubes"] + 4 * out["red_auto_mid_cubes"] + 6 * out["red_auto_top_cubes"]
    out["blue_auto_cube_points"] = 3 * out["blue_auto_bot_cubes"] + 4 * out["blue_auto_mid_cubes"] + 6 * out["blue_auto_top_cubes"]
    out["red_auto_cone_points"] = 3 * out["red_auto_bot_cones"] + 4 * out["red_auto_mid_cones"] + 6 * out["red_auto_top_cones"]
    out["blue_auto_cone_points"] = 3 * out["blue_auto_bot_cones"] + 4 * out["blue_auto_mid_cones"] + 6 * out["blue_auto_top_cones"]
    
    
    out["red_teleop_cubes"] = out["red_teleop_bot_cubes"] + out["red_teleop_mid_cubes"] + out["red_teleop_top_cubes"]
    out["blue_teleop_cubes"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_mid_cubes"] + out["blue_teleop_top_cubes"]
    out["red_teleop_cones"] = out["red_teleop_bot_cones"] + out["red_teleop_mid_cones"] + out["red_teleop_top_cones"]
    out["blue_teleop_cones"] = out["blue_teleop_bot_cones"] + out["blue_teleop_mid_cones"] + out["blue_teleop_top_cones"]
    out["red_teleop_cube_points"] = 2 * out["red_teleop_bot_cubes"] + 3 * out["red_teleop_mid_cubes"] + 5 * out["red_teleop_top_cubes"]
    out["blue_teleop_cube_points"] = 2 * out["blue_teleop_bot_cubes"] + 3 * out["blue_teleop_mid_cubes"] + 5 * out["blue_teleop_top_cubes"]
    out["red_teleop_cone_points"] = 2 * out["red_teleop_bot_cones"] + 3 * out["red_teleop_mid_cones"] + 5 * out["red_teleop_top_cones"]
    out["blue_teleop_cone_points"] = 2 * out["blue_teleop_bot_cones"] + 3 * out["blue_teleop_mid_cones"] + 5 * out["blue_teleop_top_cones"]
    
    return out

processed_matches = sorted([process_match(m, w) for m, w in all_matches if m["score_breakdown"] is not None and w is not None], key=lambda x: x["time"])
week_one_matches = [m for m in processed_matches if m["week"] == 1]

In [13]:
keys = [
    "auto_cycles",
    "teleop_cycles",
    "total_cycles",
    "auto_points",
    "teleop_points",
    "link_points",
    "total_points",
    "auto_bot_cubes",
    "auto_mid_cubes",
    "auto_top_cubes",
    "auto_bot_cones",
    "auto_mid_cones",
    "auto_top_cones",
    "auto_bot_cycles",
    "auto_mid_cycles",
    "auto_top_cycles",
    "auto_cubes",
    "auto_cones",
    "auto_cube_points",
    "auto_cone_points",
    "teleop_bot_cubes",
    "teleop_mid_cubes",
    "teleop_top_cubes",
    "teleop_bot_cones",
    "teleop_mid_cones",
    "teleop_top_cones",
    "teleop_bot_cycles",
    "teleop_mid_cycles",
    "teleop_top_cycles",
    "teleop_cubes",
    "teleop_cones",
    "teleop_cube_points",
    "teleop_cone_points",
]

team_epas = defaultdict(dict)

for key in keys:
    stats = [m["red_" + key] for m in week_one_matches] + [m["blue_" + key] for m in week_one_matches]
    mean, sd = sum(stats) / len(stats), statistics.stdev(stats)

    for team in starting_epas:
        team_epas[team][key] = [mean / 3 + sd * starting_epas[team]]

    for match in processed_matches:
        if match["playoff"]:
            continue

        red_teams = match["red"]
        blue_teams = match["blue"]

        red_pred = sum(team_epas[x][key][-1] for x in red_teams)
        blue_pred = sum(team_epas[x][key][-1] for x in blue_teams)

        red_actual = match["red_" + key]
        blue_actual = match["blue_" + key]

        red_error = red_actual - red_pred
        for team in red_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + percent * red_error / 3)

        blue_error = blue_actual - blue_pred
        for team in blue_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + percent * blue_error / 3)
        
end_epas = {
    k: {
    "country": locations[k][0],
    "state": locations[k][1],
    "district": locations[k][2],
    } 
    for k in team_epas
}

for k, v in team_epas.items():
    for k2, v2 in v.items():
        end_epas[k][k2] = v2[-1]
    
print(end_epas[254])

{'country': 'USA', 'state': 'CA', 'district': None, 'auto_cycles': 2.2102420030107646, 'teleop_cycles': 10.58280504223382, 'total_cycles': 12.735670288506093, 'auto_points': 11.98691842900499, 'teleop_points': 31.14544949644772, 'link_points': 16.140452220702617, 'total_points': 58.7677236147586, 'auto_bot_cubes': 0.4392358554399413, 'auto_mid_cubes': 0.06465478728172472, 'auto_top_cubes': 0.8830739898940088, 'auto_bot_cones': 0.06403068130355112, 'auto_mid_cones': 0.08789314779427886, 'auto_top_cones': 0.9747841595379827, 'auto_bot_cycles': 0.45351456114267064, 'auto_mid_cycles': 0.12150727756264346, 'auto_top_cycles': 1.7955928666969805, 'auto_cubes': 1.2785030926690957, 'auto_cones': 1.0266999435946322, 'auto_cube_points': 6.447429464136064, 'auto_cone_points': 6.004027301121803, 'teleop_bot_cubes': 3.013757359012539, 'teleop_mid_cubes': 0.4624030899271582, 'teleop_top_cubes': 1.7492725759179735, 'teleop_bot_cones': 0.49449609844533843, 'teleop_mid_cones': 1.3164539561603663, 'teleo

In [14]:
df = pd.DataFrame(end_epas).T

In [15]:
df.head()

,country,state,district,auto_cycles,teleop_cycles,total_cycles,auto_points,teleop_points,link_points,total_points,...,teleop_bot_cones,teleop_mid_cones,teleop_top_cones,teleop_bot_cycles,teleop_mid_cycles,teleop_top_cycles,teleop_cubes,teleop_cones,teleop_cube_points,teleop_cone_points
1,USA,MI,fim,0.875079,3.681288,4.545365,4.949726,10.07452,4.866433,19.79383,...,-0.145718,-0.170267,1.225487,1.648485,-0.225018,2.360459,2.873904,0.850156,9.081987,5.157578
4,USA,CA,None,0.912817,3.633321,4.539866,2.741662,9.654981,4.379491,16.720922,...,1.075574,0.894279,0.677121,1.323713,1.442313,0.925809,1.044563,2.613142,3.374109,8.124033
8,USA,CA,None,0.937661,4.245342,5.160951,5.763065,11.485198,3.045169,20.099308,...,0.782308,0.118231,0.668837,1.073116,1.127202,2.250753,2.88065,1.450424,11.483975,4.92752
11,USA,NJ,fma,0.973075,6.176185,7.146182,5.664309,20.702562,8.785738,35.125522,...,0.823636,0.450198,2.915484,1.354195,0.754297,4.096399,2.015429,4.172719,7.87323,17.528403
16,USA,AR,None,1.25185,5.258693,6.485824,7.566783,12.796794,9.510501,29.656475,...,0.786291,0.713636,2.020897,2.139949,0.707017,2.642339,1.96731,3.387485,5.756406,13.441363


In [20]:
df.to_json("../../backend/data/epa_breakdown.json", orient="index")